In [ ]:
import pandas as pd
df = pd.read_csv("/content/FakeNewsNet.csv")
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [ ]:
df.columns

Index(['title', 'news_url', 'source_domain', 'tweet_num', 'real'], dtype='object')

In [ ]:
df['real'].value_counts()

,count
real,
1,17441
0,5755


In [ ]:
df.isnull().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [ ]:
df.size

115980

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
#Get independent features and dependent features
X = df.drop(['real'], axis=1)
X.head()

,index,title,news_url,source_domain,tweet_num
0,0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38


In [ ]:
Y = df['real']

In [ ]:
print(X.shape, Y.shape)

(22866, 5) (22866,)


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size = 5000

In [ ]:
#one hot representation
#lets take a deep copy of the indepndent feature
messages = X.copy()
messages

,index,title,news_url,source_domain,tweet_num
0,0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38
...,...,...,...,...,...
22861,23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52
22862,23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7
22863,23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26
22864,23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24


In [ ]:
messages['title'][0]

"Kandi Burruss Explodes Over Rape Accusation on 'Real Housewives of Atlanta' Reunion (Video)"

In [ ]:
#we need to clean this title data, lower it, remove anything other than characters, and
# remove stopwords and apply porterStemmer to stem it
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stemmer = PorterStemmer()
corpus = []
for word in range(0, len(messages)):
  message = re.sub('[^a-zA-Z]',' ', messages['title'][word])
  message = message.lower()
  message = message.split()
  message = [stemmer.stem(word) for word in message if word not in stopwords.words('english')]
  message = ' '.join(message)
  corpus.append(message)
corpus

['kandi burruss explod rape accus real housew atlanta reunion video',
 'peopl choic award best red carpet look',
 'sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva',
 'colombian singer maluma spark rumour inappropri relationship aunt',
 'gossip girl year later upper east sider shock world chang pop cultur forev',
 'gwen stefani got dump blake shelton jealousi drama exclus',
 'broward counti sheriff fire lie parkland',
 'amber rose shut french montana date rumor call rapper bruvaaa',
 'mindi kale make first post babi appear disneyland wrinkl time co star',
 'katharin mcphee butcher toni nomin drink',
 'wag miami star ashley nicol robert philip wheeler marri',
 'mel gibson hollywood pedophil nowher left hide',
 'medium tyler henri address chill messag kristin cavallari deceas brother express read',
 'dwt season result week disney night',
 'reason tarek el moussa overcom latest back injuri',
 'david cassidi cut estrang daughter kati complet leav son b

In [ ]:
#now we need to convert each sentence in corpus to one hot encoding
onehot_repr = [one_hot(sent, voc_size) for sent in corpus]
# len(onehot_repr)
onehot_repr

[[121, 2695, 337, 4800, 4562, 2139, 4694, 4366, 1043, 4003],
 [4642, 3331, 3932, 1652, 815, 1847, 924],
 [1180,
  1879,
  4600,
  4943,
  2299,
  4097,
  2637,
  4978,
  762,
  3135,
  3599,
  927,
  2847,
  2215,
  3595],
 [2859, 4883, 1967, 4562, 2034, 4246, 3391, 3157],
 [3560,
  4313,
  2908,
  1348,
  3260,
  2836,
  1604,
  1773,
  2989,
  3898,
  3081,
  3508,
  1160],
 [278, 3580, 418, 1252, 4065, 4962, 2904, 3198, 3846],
 [129, 797, 2817, 1665, 1726, 3252],
 [2027, 1146, 4778, 1404, 3334, 3668, 17, 1513, 3080, 1899],
 [1115, 2914, 3377, 1744, 2023, 4981, 3030, 851, 928, 1081, 3135, 3599],
 [3300, 3722, 3007, 2224, 3197, 2062],
 [602, 385, 3599, 423, 2144, 4294, 1489, 614, 2119],
 [3365, 999, 3024, 2108, 3202, 845, 3844],
 [374, 1661, 4413, 1182, 2820, 4097, 35, 1268, 4841, 3852, 2985, 4289],
 [2552, 1840, 2036, 3213, 3205, 3676],
 [4734, 753, 2132, 2666, 4914, 3695, 1951, 157],
 [4708, 4754, 1199, 243, 4103, 4653, 3523, 4077, 2243, 3533],
 [3049, 2888, 1100, 2315, 1810, 834, 1

In [ ]:
#we need to either perform pre or post padding
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4366, 1043, 4003],
       [   0,    0,    0, ...,  815, 1847,  924],
       [   0,    0,    0, ..., 2847, 2215, 3595],
       ...,
       [   0,    0,    0, ..., 2707,  225, 2738],
       [   0,    0,    0, ..., 3350, 4356, 3846],
       [   0,    0,    0, ..., 4695, 1634, 3932]], dtype=int32)

In [ ]:
#model
from tensorflow.keras.layers import Dropout
dim = 40
model = Sequential()
model.add(Embedding(voc_size, dim, input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#final inputs
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)
print(X_final.shape, Y_final.shape)

(22866, 20) (22866,)


In [ ]:
#train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

In [ ]:
#model training
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7694 - loss: 0.5441 - val_accuracy: 0.8328 - val_loss: 0.4020
Epoch 2/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8458 - loss: 0.3522 - val_accuracy: 0.8325 - val_loss: 0.3886
Epoch 3/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8598 - loss: 0.3153 - val_accuracy: 0.8305 - val_loss: 0.3950
Epoch 4/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8817 - loss: 0.2762 - val_accuracy: 0.8146 - val_loss: 0.4242
Epoch 5/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8902 - loss: 0.2635 - val_accuracy: 0.8275 - val_loss: 0.4279
Epoch 6/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9000 - loss: 0.2353 - val_accuracy: 0.8220 - val_loss: 0.4576
Epoch 7/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9124 - loss: 0.2093 - val_accuracy: 0.8235 - val_loss: 0.5028
Epoch 8/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9137 - loss: 0.1997 - val_accuracy: 0

In [ ]:
y_pred = model.predict(x_test)
y_pred

236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


array([[0.7549014 ],
       [0.99993443],
       [0.9983311 ],
       ...,
       [0.01362295],
       [0.9994567 ],
       [0.98366874]], dtype=float32)

In [ ]:
y_pred = np.where(y_pred>0.5,1,0)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
accuracy_score(y_test, y_pred)

0.8199045852107076

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.56      0.60      1842
           1       0.86      0.90      0.88      5704

    accuracy                           0.82      7546
   macro avg       0.76      0.73      0.74      7546
weighted avg       0.81      0.82      0.81      7546



In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[1030  812]
 [ 547 5157]]


In [ ]:
#now lets train the model with bidirectional LSTM
#model
from tensorflow.keras.layers import Dropout, Bidirectional
dim = 40
model = Sequential()
model.add(Embedding(voc_size, dim, input_length = sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#model training
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7612 - loss: 0.5354 - val_accuracy: 0.8297 - val_loss: 0.3970
Epoch 2/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8416 - loss: 0.3570 - val_accuracy: 0.8268 - val_loss: 0.3924
Epoch 3/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8687 - loss: 0.3085 - val_accuracy: 0.8359 - val_loss: 0.3978
Epoch 4/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8846 - loss: 0.2794 - val_accuracy: 0.8317 - val_loss: 0.4223
Epoch 5/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9000 - loss: 0.2459 - val_accuracy: 0.8269 - val_loss: 0.4330
Epoch 6/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9099 - loss: 0.2238 - val_accuracy: 0.8154 - val_loss: 0.4602
Epoch 7/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9168 - loss: 0.2106 - val_accuracy: 0.8188 - val_loss: 0.4866
Epoch 8/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9240 - loss: 0.1918 - val_accuracy: